In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html

import torch
import torch.nn.functional as F
from torch import nn
import tqdm
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.multioutput import MultiOutputClassifier
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL

from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader, LinkNeighborLoader
from torch_geometric.nn import GraphSAGE
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T

dataset = TUDataset(root='/tmp/NCI1', name='NCI1', transform=T.NormalizeFeatures())
torch.manual_seed(12315)
dataset = dataset.shuffle()
dataset_length = len(dataset)

# split the dataset into 3 parts
DA_train = dataset[0:int(0.4*dataset_length)]
D_aux = dataset[int(0.4*dataset_length):int(0.7*dataset_length)]
DA_test = dataset[int(0.7*dataset_length):]

buck_num = 8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu116.html
     |████████████████████████████████| 8.0 MB 73.2 MB/s 
     |████████████████████████████████| 3.5 MB 338 kB/s 
     |████████████████████████████████| 2.4 MB 302 kB/s 
     |████████████████████████████████| 706 kB 334 kB/s 
     |████████████████████████████████| 467 kB 7.5 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689857 sha256=43cd609a75b451a3a29fcd792ebbbd517c1520a58f6e8b9ba43d2d6f15b888ee
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric


Extracting /tmp/NCI1/NCI1/NCI1.zip
Processing...
Done!


# Load the model

In [ ]:
model_path = "NCI_model.pt"

model_sage = torch.load(model_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_sage = model_sage.to(device)

# Get graph embedding for D_aux

# Get labels for D_aux

# for num of nodes:

# for num of edges:

In [ ]:
# get graph embedding from node embedding for D aux using mean pool
D_aux_graph_embedding = []
D_aux_num_of_nodes_raw_data = []
D_aux_num_of_edges_raw_data = []

for data in D_aux:
    data = data.to(device)
    test_output = model_sage(data.x, data.edge_index)
    D_aux_graph_embedding.append(test_output.sum(dim=0).cpu())
    D_aux_num_of_nodes_raw_data.append(data.num_nodes)
    D_aux_num_of_edges_raw_data.append(data.num_edges)

del test_output
del data

# split data for train and validation

In [ ]:
print(len(D_aux_num_of_nodes_raw_data))

D_aux_num_of_nodes_raw_data_train = D_aux_num_of_nodes_raw_data[0:int(0.7*len(D_aux_num_of_nodes_raw_data))]
D_aux_num_of_edges_raw_data_train = D_aux_num_of_edges_raw_data[0:int(0.7*len(D_aux_num_of_edges_raw_data))]
D_aux_graph_embedding_train = D_aux_graph_embedding[0:int(0.7*len(D_aux_graph_embedding))]
print(len(D_aux_num_of_nodes_raw_data_train))

D_aux_num_of_nodes_raw_data_test = D_aux_num_of_nodes_raw_data[int(0.7*len(D_aux_num_of_nodes_raw_data)):]
D_aux_num_of_edges_raw_data_test = D_aux_num_of_edges_raw_data[int(0.7*len(D_aux_num_of_edges_raw_data)):]
D_aux_graph_embedding_test = D_aux_graph_embedding[int(0.7*len(D_aux_graph_embedding)):]
print(len(D_aux_num_of_nodes_raw_data_test))

1233
863
370


# quantile bucketing


In [ ]:
def split_buck(data, buck):
    # get the maximum number in data
    N_max = data[0]

    for i in data:        
        if i > N_max:
            N_max = i
            
    # sort the data w.r.t. data.num_nodes
    sorted_list = sorted(data, key=lambda x: x, reverse=False)
    # print('length of sorted list: ', len(sorted_list))
    
    # num of elements in each buck
    buck_num = round(len(data) / buck)
    
    # split point for data w.r.t. buck
    split_pt = []
    cnt = 0
    for i in sorted_list:
        cnt += 1
        if cnt == buck_num:
            cnt = 0
            split_pt.append(i)
    
    if len(split_pt) == buck:
        split_pt[-1] = N_max + 1
    else:
        split_pt.append(N_max + 1)
        
    res = []
    for i in data:
        for index, j in enumerate(split_pt):
            if i <= j:
                res.append(index)
                break
    
    return res

In [ ]:
D_aux_nodes_train = split_buck(D_aux_num_of_nodes_raw_data_train, buck_num)
D_aux_nodes_test = split_buck(D_aux_num_of_nodes_raw_data_test, buck_num)

D_aux_edges_train = split_buck(D_aux_num_of_edges_raw_data_train, buck_num)
D_aux_edges_test = split_buck(D_aux_num_of_edges_raw_data_test, buck_num)

In [ ]:
class TrainSet(Dataset):
    def __init__(self, X, num_nodes, num_edges):
        self.X = torch.stack(X)
        self.num_nodes = torch.tensor(num_nodes, dtype=torch.float)
        self.num_edges = torch.tensor(num_edges, dtype=torch.float)

    def __getitem__(self, index):
        return self.X[index], self.num_nodes[index], self.num_edges[index]

    def __len__(self):
        return len(self.num_nodes)

In [ ]:
# mydataset = TrainSet(D_aux_graph_embedding_train, D_aux_nodes_train)
mydataset = TrainSet(D_aux_graph_embedding_train, D_aux_nodes_train, D_aux_edges_train)
train_loader = DL(mydataset, batch_size=10, shuffle=True)

# Define the multi-task learning model

# sharing the feature net, and has its own classification net at the end for each task

In [ ]:
class Network(nn.Module):
    def __init__(self, buck_num):
        super().__init__()
        self.num_features = 192
        
        self.num_buck = buck_num
        
        self.featureNet = nn.Sequential(
            nn.Linear(self.num_features, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, self.num_features),
            nn.ReLU()
        )
        
        self.nodeNet = nn.Sequential(
            nn.Linear(self.num_features, self.num_features),
            nn.ReLU(),
            nn.Linear(self.num_features, self.num_buck)
        )
        
        self.edgeNet = nn.Sequential(
            nn.Linear(self.num_features, self.num_features),
            nn.ReLU(),
            nn.Linear(self.num_features, self.num_buck)
        )
        
    def forward(self, x):
        x = self.featureNet(x)
        pred_nodes = self.nodeNet(x)
        pred_edges = self.edgeNet(x)
        
        return pred_nodes, pred_edges

# class Network(nn.Module):
#     def __init__(self, buck_num):
#         super().__init__()
        
#         self.num_features = 192
        
#         self.linear1 = nn.Linear(self.num_features, 256)
#         self.linear2 = nn.Linear(256, 256)
#         self.linear3 = nn.Linear(256, self.num_features)
#         self.linear4 = nn.Linear(self.num_features, buck_num)
        
#     def forward(self, x):
#         x = F.relu(self.linear1(x))
#         x = F.relu(self.linear2(x))
#         x = F.relu(self.linear3(x))
#         x = self.linear4(x)
        
#         return x

In [ ]:
model = Network(buck_num).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
def train():
    model.train()
    total_loss = 0
    
    for data in tqdm.tqdm(train_loader):
        inputs, num_nodes, num_edges = data
        optimizer.zero_grad()
        
        inputs = inputs.detach()
        num_nodes = num_nodes.detach()
        num_edges = num_edges.detach()
        
        pred_nodes, pred_edges = model(inputs.to(device))
        loss_1 = criterion(pred_nodes, num_nodes.type(torch.LongTensor).to(device))
        loss_2 = criterion(pred_edges, num_edges.type(torch.LongTensor).to(device))
        
        loss = loss_1 + loss_2
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(D_aux_nodes_train)
        

# def train():
#     model.train()
#     total_loss = 0
    
#     for data in tqdm.tqdm(train_loader):
#         inputs, num_node = data        
#         optimizer.zero_grad()
#         # !IMPORTANT
#         # need to extract the data first
#         inputs = inputs.detach()
#         num_node = num_node.detach()
        
#         pred_node = model(inputs.to(device))
#         loss = criterion(pred_node, num_node.type(torch.LongTensor).to(device))
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
        
#     return total_loss / len(D_aux_nodes_train)

In [ ]:
# mytestdata = TrainSet(D_aux_graph_embedding_test, D_aux_nodes_test)
mytestdata = TrainSet(D_aux_graph_embedding_test, D_aux_nodes_test, D_aux_edges_test)
test_loader = DL(mytestdata, batch_size=1, shuffle=False)

In [ ]:
@torch.no_grad()
def test():
    model.eval()
    total_loss = 0
    accu_nodes = 0
    accu_edges = 0
    
    for data in tqdm.tqdm(test_loader):
        inputs, num_nodes, num_edges = data
        inputs = inputs.detach()
        num_nodes = num_nodes.detach()
        num_edges = num_edges.detach()
        
        pred_nodes, pred_edges = model(inputs.to(device))
        
        pred_nodes_argmax = torch.argmax(pred_nodes, dim=1).type(torch.float).cpu()
        pred_edges_argmax = torch.argmax(pred_edges, dim=1).type(torch.float).cpu()
        
        if (torch.equal(pred_nodes_argmax, num_nodes)):
            accu_nodes += 1
        
        if (torch.equal(pred_edges_argmax, num_edges)):
            accu_edges += 1
        
        loss_1 = criterion(pred_nodes, num_nodes.type(torch.LongTensor).to(device))
        
        loss_2 = criterion(pred_edges, num_edges.type(torch.LongTensor).to(device))
        
        loss = loss_1 + loss_2

        total_loss += loss.item()
        
    return total_loss / len(D_aux_nodes_test), accu_nodes / len(D_aux_nodes_test), accu_edges / len(D_aux_nodes_test)

# @torch.no_grad()
# def test():
#     model.eval()
#     total_loss = 0
#     accu = 0
    
#     for data in tqdm.tqdm(test_loader):
#         inputs, num_node = data
#         inputs = inputs.detach()
#         num_node = num_node.detach()
        
#         pred_node = model(inputs.to(device))
#         # print(pred_node.shape)  1x8
#         # print(num_node)
#         argmax = torch.argmax(pred_node, dim=1).type(torch.float).cpu()
#         # print(argmax)
#         if (torch.equal(argmax, num_node)):
#             accu += 1
        
#         loss = criterion(pred_node, num_node.type(torch.LongTensor).to(device))
#         total_loss += loss.item()
    
#     return total_loss / len(D_aux_nodes_test), accu / len(D_aux_nodes_test)

In [ ]:
for epoch in range(60):
    train_loss = train()
    test_loss, test_nodes_accu, test_edges_accu = test()
    print(f'Epoch: {epoch:4d}, Train loss: {train_loss:.4f}, Test loss: {test_loss:.4f}, Test node accu: {test_nodes_accu:.4f}, Test edge accu: {test_edges_accu:.4f}')

100%|██████████| 370/370 [00:00<00:00, 1149.89it/s]


Epoch:    0, Train loss: 0.4093, Test loss: 4.0236, Test node accu: 0.1432, Test edge accu: 0.1486


100%|██████████| 370/370 [00:00<00:00, 1152.06it/s]


Epoch:    1, Train loss: 0.3936, Test loss: 3.9772, Test node accu: 0.1730, Test edge accu: 0.1838


100%|██████████| 370/370 [00:00<00:00, 1192.11it/s]


...

Epoch:   57, Train loss: 0.0908, Test loss: 3.6604, Test node accu: 0.4270, Test edge accu: 0.4622


100%|██████████| 370/370 [00:00<00:00, 1314.33it/s]


Epoch:   58, Train loss: 0.0954, Test loss: 4.4064, Test node accu: 0.3459, Test edge accu: 0.4000


100%|██████████| 370/370 [00:00<00:00, 1306.90it/s]

Epoch:   59, Train loss: 0.0993, Test loss: 3.4746, Test node accu: 0.4486, Test edge accu: 0.4649


In [ ]:
a = torch.rand((1,8))
print(a)
b = torch.argmax(a, dim=1)

tensor([[0.0983, 0.7518, 0.1657, 0.4712, 0.6338, 0.2363, 0.5027, 0.2240]])


In [ ]:
c = torch.tensor([6.])

In [ ]:
c

tensor([6.])

In [ ]:
b.type(torch.float)

tensor([1.])

In [ ]:
(b == c) is True

False

In [ ]:
torch.equal(b.type(torch.float), c)

False